In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [9]:
path0='C:\\Users\\ADMIN\AppData\Roaming\MetaQuotes\Terminal\\2506E8E7E4116548D478CE2C3598FAB1\MQL5\Files\shakhS kl6-D1.prn'
path1='C:\\Users\\ADMIN\AppData\Roaming\MetaQuotes\Terminal\\2506E8E7E4116548D478CE2C3598FAB1\MQL5\Files\\USDIRR-D1.prn'
path2='C:\\Users\\ADMIN\AppData\Roaming\MetaQuotes\Terminal\\2506E8E7E4116548D478CE2C3598FAB1\MQL5\Files\\skh-jdid-D1.prn'
path3='C:\\Users\\ADMIN\AppData\Roaming\MetaQuotes\Terminal\\D0E8209F77C8CF37AD8BF550E51FF075\MQL5\Files\\XAUUSD-D1.prn'
path4='C:\\Users\\ADMIN\AppData\Roaming\MetaQuotes\Terminal\\D0E8209F77C8CF37AD8BF550E51FF075\MQL5\Files\\BTCUSD-D1.prn'
files=[path0,path1,path2,path3,path4]

In [10]:
path1.split('\\')[-1]

'USDIRR-D1.prn'

In [11]:
def mean_exp(data_in):
    p = 1
    n = 0
    for i in data_in:
        if i != 0:
            p*=i
            n+=1
    return p**(1/n)

In [16]:
for K in files:
    data_df=pd.read_csv(K)
    date=data_df['<DTYYYYMMDD>']
    vol=data_df['<VOL>']
    data_df.drop(['<DTYYYYMMDD>','<TIME>','<VOL>'],axis=1,inplace=True)
    col=data_df.columns
    data_df.rename(columns={'<OPEN>':'open','<HIGH>':'high','<LOW>':'low','<CLOSE>':'close'},inplace=True)
    mapper={i:f'{i}_return_1d' for i in data_df.columns}
    data_exp={f'{mapper[i]}':[1] for i in mapper.keys()}
    for i in data_df.columns:
        for j in data_df.index[1:]:
            data_exp[mapper[i]].append(data_df[i][j]/data_df[i][j-1])
    data2=pd.DataFrame(data_exp)
    mean_col={}
    for i in data_df.columns:
        mean_col[f'{i}']=mean_exp(data2[f'{i}_return_1d'])
    expected_data={}
    for i in data_df.columns:
        expected_data[f'exp_{i}']=[]
        for j in data_df.index:
            expected_data[f'exp_{i}'].append(data_df[i][0]*mean_col[i]**j)
    expected_data=pd.DataFrame(expected_data)
    final_data=pd.concat([date,expected_data],axis=1)
    final_data2=data_df.values/expected_data.values
    final_data2=pd.DataFrame(final_data2)
    for_export_data=pd.concat([date,final_data2,vol],axis=1)
    for_export_data.rename(columns={0:'<OPEN>',1:'<HIGH>',2:'<LOW>',3:'<CLOSE>'},inplace=True)
    plt.plot(range(len(for_export_data.index)),for_export_data['<CLOSE>'],c='blue')
    plt.plot([for_export_data['<CLOSE>'].mean()]*len(for_export_data.index),c='red')
    plt.legend(['Close','Mean_return'])
    plt.text(0,for_export_data['<CLOSE>'].mean(),f"{for_export_data['<CLOSE>'].mean()}",c='black')
    plt.title(K.split('\\')[-1][-4])
    plt.grid()
    plt.show()
    file=open('C:\\Users\ADMIN\Desktop\\tse\\'+'Expected-'+K.split('\\')[-1],'w')
    file.write('<DTYYYYMMDD>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>')
    file.write('\n')
    for i in for_export_data.index:
        for j in for_export_data.columns:
            if j != for_export_data.columns[-1]:
                file.write(str(for_export_data[j][i]))
                file.write(',')
            else:
                file.write(str(for_export_data[j][i]))
        file.write('\n')
    file.close()



'1'